In [219]:
import pandas as pd
import sklearn
train = pd.read_csv("train.tsv")
test = pd.read_csv("test.tsv")
sample_submission = pd.read_csv("sample_submission.tsv")
import numpy as np

In [220]:
import xgboost
cols = train.ix[:, "f31":].columns
train.drop(cols, 1, inplace=True)
test.drop(cols, 1, inplace=True)

In [221]:
data = test
ans = []
for i, row in data.iterrows():
    new_week = row["week"] - row["shift"]
    new_year = row["year"]
    if new_week < 1:
        new_week += 52
        new_year -= 1
    index = (data["year"] == new_year) & (data["week"] == new_week) & (data["item_id"] == row["item_id"])
    if index.any():
        data.loc[index, "y"] = data.loc[i, "f30"] * 1.60998758761
        

In [222]:
data = pd.concat([train, test])

In [223]:
data = data.dropna()
data.loc[:,"y"] = np.log(data["y"] + 1)

In [224]:
X = data.drop(["Num", "y"], axis=1)
y = data["y"]

In [174]:
def smape(a, b):
    a = np.exp(a) - 1
    b = np.exp(b) - 1
    return np.mean(np.abs(a - b) / (np.abs(a) + np.abs(b)))
smape = sklearn.metrics.make_scorer(smape, False)

In [227]:
sklearn.model_selection.cross_val_score(xgboost.XGBRegressor(max_depth=10, n_estimators=100), X, y, scoring=smape)

array([-0.13087068, -0.12993722, -0.1444636 ])

In [203]:
gs = sklearn.model_selection.RandomizedSearchCV(model, {"max_depth": [3, 4, 5, 6], "n_estimators": [200, 300, 500, 800, 1300]}, smape)

In [153]:
gs.fit(X, y)

GridSearchCV(cv=None, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=2,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_depth': [2, 3, 4]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True,
       scoring=make_scorer(smape, greater_is_better=False), verbose=0)

In [154]:
gs.best_params_

{'max_depth': 4}

In [160]:
# Final results

In [208]:
model = xgboost.XGBRegressor(max_depth=4, n_estimators=100)
model.fit(X, y)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [205]:
X_test = test[data.drop(["y"], axis=1).columns].drop(['Num'], axis=1)

In [206]:
preds = model.predict(X_test)

In [202]:
for i, row in test.iterrows():
    index = (data["year"] == row["year"]) & (data["week"] == row["week"]) & (data["item_id"] == row["item_id"])
    if index.any():
        preds[i] = data.loc[index, "y"].iloc[0]
preds = np.exp(preds) - 1

In [199]:
sample_submission['y'] = preds
sample_submission['y'] = sample_submission['y'].map(lambda x: x if x > 0 else 0.0)
sample_submission.to_csv("baseline_submission.tsv", sep=',', index=False)

-0.075937770307064056